In [1]:
import pandas as pd
import numpy as np
import gseapy as gp

gmt_file = '/Users/Miko/Desktop/CCBB/Network/gmt/c2.cp.v6.1.symbols.gmt'
expression_file = '/Users/Miko/Desktop/CCBB/Network/mouse_liver/exp_ALCvsCHOWinNTinAlbCre.tsv'
meta_file = "/Users/Miko/Desktop/CCBB/Network/mouse_liver/dHEP_metadata.csv" # more exhaustive table, not all needed
output_dir = '/Users/Miko/Desktop/CCBB/Network/output/mouse_liver_2'

### Load in meta_file as a dataframe

In [2]:
df_meta = pd.read_csv(meta_file)
# This also works:  df_meta = pd.read_table(meta_file, sep=',') # use sep (separator) if read_table
df_meta.head()

,Sample_name,Sample_Name2,mouse genotype,short_genotype,Model,Treatment,Tissue,Sample order in the expression file
0,dHEP_518_A_NT,dHEP-518-A-NT,Alb-Cre;IL-17RA-flox/flox,AlbCre_IL17RA_floxflox,DEN,DEN_HFD_alcohol,Liver,10.0
1,dHEP_520_A_NT,dHEP-520-A-NT,Alb-Cre;IL-17RA-flox/flox,AlbCre_IL17RA_floxflox,DEN,DEN_HFD_alcohol,Liver,1.0
2,dHEP_549_A_NT,dHEP-549-A-NT,Alb-Cre;IL-17RA-flox/flox,AlbCre_IL17RA_floxflox,DEN,DEN_HFD_alcohol,Liver,4.0
3,dHEP_550_A_NT,dHEP-550-A-NT,Alb-Cre;IL-17RA-flox/flox,AlbCre_IL17RA_floxflox,DEN,DEN_HFD_alcohol,Liver,2.0
4,F_F_503_A_NT,F/F-503-A-NT,IL-17RA-flox/flox,IL17RA_floxflox,DEN,DEN_HFD_alcohol,Liver,5.0


### Parse df_meta, only keeping the samples used in the experiment
### Create a samp_to_class dataframe as cls, from df_meta

In [3]:
# Note:  df_meta['Tissue'] == 'Liver' # this prints true or false for each row

# only keep the rows with the related samples
df_meta = df_meta[df_meta['Tissue'] == 'Liver']

# extract only the columns with sample_name and treatment
samp_to_class = df_meta[['Sample_name','Treatment']]

# only keep the rows with DEN_HFD_alcohol or DEN_only treatment (delete "no injury")
samp_to_class = samp_to_class[(samp_to_class['Treatment'] == 'DEN_HFD_alcohol') | (samp_to_class['Treatment'] == 'DEN_only')]

print(len(samp_to_class))
samp_to_class

14


,Sample_name,Treatment
0,dHEP_518_A_NT,DEN_HFD_alcohol
1,dHEP_520_A_NT,DEN_HFD_alcohol
2,dHEP_549_A_NT,DEN_HFD_alcohol
3,dHEP_550_A_NT,DEN_HFD_alcohol
4,F_F_503_A_NT,DEN_HFD_alcohol
5,F_F_523_A_NT,DEN_HFD_alcohol
6,F_F_581_A_NT,DEN_HFD_alcohol
7,F_F_595_A_NT,DEN_HFD_alcohol
8,dHEP_155_NC_NT,DEN_only
9,dHEP_164_NC_NT,DEN_only


In [ ]:
type(samp_to_class)

### Get a list of sample names

In [4]:
focal_samples = samp_to_class['Sample_name'].tolist()
focal_samples

['dHEP_518_A_NT',
 'dHEP_520_A_NT',
 'dHEP_549_A_NT',
 'dHEP_550_A_NT',
 'F_F_503_A_NT',
 'F_F_523_A_NT',
 'F_F_581_A_NT',
 'F_F_595_A_NT',
 'dHEP_155_NC_NT',
 'dHEP_164_NC_NT',
 'dHEP_165_NC_NT',
 'F_F_157_NC_NT',
 'F_F_158_NC_NT',
 'F_F_159_NC_NT']

In [ ]:
len(focal_samples)

### Load expression file, use the gene names as row labels

In [5]:
#df_expression = pd.read_table(expression_file)
df_expression = pd.read_table(expression_file, index_col='Unnamed: 0')
df_expression.head()

,dHEP_520_A_NT,dHEP_550_A_NT,F_F_158_NC_NT,dHEP_549_A_NT,F_F_503_A_NT,dHEP_165_NC_NT,F_F_595_A_NT,F_F_523_A_NT,F_F_157_NC_NT,dHEP_518_A_NT,dHEP_164_NC_NT,dHEP_155_NC_NT,F_F_159_NC_NT,F_F_581_A_NT
Gnai3,6.482501,6.648272,6.644257,6.734103,6.557685,6.672892,6.707925,6.591240,6.632168,6.483054,6.815480,6.667075,6.768433,6.708067
Cdc45,0.779113,0.881496,0.026606,0.036682,0.688223,0.222011,0.281181,0.493432,0.192387,1.325266,1.160785,0.731233,0.930572,0.931043
Apoh,10.437362,10.285868,10.683067,10.462445,10.204836,10.558875,10.289569,10.032467,10.643058,10.364403,10.533293,10.551628,10.831810,10.346315
Narf,5.990224,6.089289,5.152588,6.280477,6.093053,6.277727,6.377708,5.902462,5.912509,6.144837,6.152632,5.551067,5.925389,6.137725
Cav2,1.732318,2.221725,2.069250,2.199621,1.634452,2.148011,2.390303,2.233464,1.226334,1.180876,2.263471,2.416598,1.768019,2.586876


In [6]:
df_expression.shape # returns a tuple representing the dimensionality of the DataFrame.

(12651, 14)

### Process the df_expression, to be in the same order as focal_samples
### Note: the length of focal_samples must EQUAL the number of columns in df_expression

In [7]:
df_expression = df_expression[focal_samples]
df_expression.head()

,dHEP_518_A_NT,dHEP_520_A_NT,dHEP_549_A_NT,dHEP_550_A_NT,F_F_503_A_NT,F_F_523_A_NT,F_F_581_A_NT,F_F_595_A_NT,dHEP_155_NC_NT,dHEP_164_NC_NT,dHEP_165_NC_NT,F_F_157_NC_NT,F_F_158_NC_NT,F_F_159_NC_NT
Gnai3,6.483054,6.482501,6.734103,6.648272,6.557685,6.591240,6.708067,6.707925,6.667075,6.815480,6.672892,6.632168,6.644257,6.768433
Cdc45,1.325266,0.779113,0.036682,0.881496,0.688223,0.493432,0.931043,0.281181,0.731233,1.160785,0.222011,0.192387,0.026606,0.930572
Apoh,10.364403,10.437362,10.462445,10.285868,10.204836,10.032467,10.346315,10.289569,10.551628,10.533293,10.558875,10.643058,10.683067,10.831810
Narf,6.144837,5.990224,6.280477,6.089289,6.093053,5.902462,6.137725,6.377708,5.551067,6.152632,6.277727,5.912509,5.152588,5.925389
Cav2,1.180876,1.732318,2.199621,2.221725,1.634452,2.233464,2.586876,2.390303,2.416598,2.263471,2.148011,1.226334,2.069250,1.768019


### Addin a Name column, using capitalized gene names

In [8]:
cap_gene = [str(g).upper() for g in df_expression.index.tolist()] # cap the genes
df_expression['Name'] = cap_gene                                  # create a new column
df_expression = df_expression[['Name'] + focal_samples]           # put the 'Name' column at front
df_expression.index = range(0,len(df_expression))                 # number the rows
df_expression.head()

,Name,dHEP_518_A_NT,dHEP_520_A_NT,dHEP_549_A_NT,dHEP_550_A_NT,F_F_503_A_NT,F_F_523_A_NT,F_F_581_A_NT,F_F_595_A_NT,dHEP_155_NC_NT,dHEP_164_NC_NT,dHEP_165_NC_NT,F_F_157_NC_NT,F_F_158_NC_NT,F_F_159_NC_NT
0,GNAI3,6.483054,6.482501,6.734103,6.648272,6.557685,6.591240,6.708067,6.707925,6.667075,6.815480,6.672892,6.632168,6.644257,6.768433
1,CDC45,1.325266,0.779113,0.036682,0.881496,0.688223,0.493432,0.931043,0.281181,0.731233,1.160785,0.222011,0.192387,0.026606,0.930572
2,APOH,10.364403,10.437362,10.462445,10.285868,10.204836,10.032467,10.346315,10.289569,10.551628,10.533293,10.558875,10.643058,10.683067,10.831810
3,NARF,6.144837,5.990224,6.280477,6.089289,6.093053,5.902462,6.137725,6.377708,5.551067,6.152632,6.277727,5.912509,5.152588,5.925389
4,CAV2,1.180876,1.732318,2.199621,2.221725,1.634452,2.233464,2.586876,2.390303,2.416598,2.263471,2.148011,1.226334,2.069250,1.768019


### Now, df_expression and samp_to_class match each other by sample names

In [9]:
samp_to_class

,Sample_name,Treatment
0,dHEP_518_A_NT,DEN_HFD_alcohol
1,dHEP_520_A_NT,DEN_HFD_alcohol
2,dHEP_549_A_NT,DEN_HFD_alcohol
3,dHEP_550_A_NT,DEN_HFD_alcohol
4,F_F_503_A_NT,DEN_HFD_alcohol
5,F_F_523_A_NT,DEN_HFD_alcohol
6,F_F_581_A_NT,DEN_HFD_alcohol
7,F_F_595_A_NT,DEN_HFD_alcohol
8,dHEP_155_NC_NT,DEN_only
9,dHEP_164_NC_NT,DEN_only


### Run GSEA

In [10]:
gs_res = gp.gsea(data=df_expression, 
                 gene_sets=gmt_file,
                 cls=samp_to_class['Treatment'].tolist(),  # we only need Treatment column here, since the Sample_name is in the expression file
                 permutation_num=100, # reduce number to speed up test
                 weighted_score_type = 0,
                 outdir=output_dir,
                 method='ratio_of_classes',
                 processes=4,    ## 1 is default
                 format='png')

2018-02-15 14:03:08,852 Warning: dropping duplicated gene names, only keep the first values


In [11]:
# look up gsea info
gp.gsea?

In [ ]:
#access the dataframe results throught res2d attribute
gs_res.res2d.head()

In [ ]:
# plotting
gsea_results= gs_res.res2d
with plt.style.context('ggplot'):
    gsea_results = gsea_results.reset_index()
    gsea_results.head(5).plot.barh(y='fdr',x='Term',fontsize=16)